# Engenharia de Prompt

Engenharia de prompt é uma disciplina relativamente nova que aborda o desenvolvimento e a otimização de prompts para o uso eficiente de modelos de linguagem (LMs) em uma ampla variedade de aplicativos e tópcios de pesquisa. Os conhecimentos adquiridos através do estudo da Engenharia de Prompt ajudam a entender melhor as capacidades e limitações dos modelos extensos de linguagem (LLMs).

Os pesquisadores utilizam engenharia de prompt para melhorar a capacidade dos LLMs em uma ampla gama de tarefas complexas como question & answering e raciocínio aritmético. Desenvolvedores utilizam a engenharia de prompt para desenhar técnicas de prompt robutas e efetivas que relacionam LLMs e outras ferramentas.

É uma habilidade importante para interagir, construir e entender os recursos dos LLMs. Você pode usar a engenharia imediata para melhorar a segurança dos LLMs e criar novos recursos, como aumentar os LLMs com conhecimento de domínio e ferramentas externas.

## Introdução

Este guia cobre o básico sobre prompts a fim de prover uma ideia simplificada de como usar os prompts para interagir com modelos de linguagem.

### Configurações dos Modelos de Lingugem

Quando se trabalha com propmts, você pode interagir com o modelo através de uma interface ou de uma API. Pode-se configurar alguns parâmetros para obter diferentes resultados a partir dos prompts:

- ```temperature```: em resumo, quanto menor a temperatura, mais determinístico é o resultado no sentido de que é mais provável que o próximo token será sempre escolhido. Aumentar a temperatura pode elevar a aleatoriedade, o que produz saídas mais diversas ou criativas. É essencialmente elevar os pesos dos outros tokens possíveis. Em termos de aplicação, você deve querer usar valores baixos de temperatura para tarefas como QA baseado em fatos para encorajar respostas mais concisas. Para geração de poemas ou outras atividades mais criativas, deve ser benéfico usar valores mais altos de temperatura.

- ```top_p```: de forma semelhante, com uma técnica de amostragem chamada nucleus sampling, você pode controlar o quão determinístico o modelo é ao gerar uma resposta. Se estiver procurando por respostas exatas e factuais, mantenha o top_p baixo. Para respostas mais diversas, aumente esse valor.

A recomendação geral é alterar apenas um desses parâmetros, nunca ambos.

### Elementos do Prompt

Há certos elementos que compõem os prompts:

- **Instrução**: uma tarefa ou instrução específica que o modelo deve executar
- **Contexto**: informação externa ou contexto adicional que guia o modelo para respostas melhores
- **Dados de entrada**: a entrada ou questão para a qual a resposta é requisitada
- **Indicador de saída**: o tipo ou formato da resposta

#### Célula de configuração

Abaixo iremos construir os blocos de código necessários para testar os prompts.

In [1]:
import openai
import configparser

config = configparser.ConfigParser()
config.read('config.ini') # caminho do arquivo de config .ini

openai.api_key = config['AZURE_CONFIG']['AZURE_OPENAI_KEY']
openai.api_base = config["AZURE_CONFIG"]["AZURE_OPENAI_ENDPOINT"]
openai.api_type = 'azure'
openai.api_version = '2023-05-15' # pode mudar futuramente
DEPLOYMENT_NAME = config["AZURE_CONFIG"]["DEPLOYMENT_NAME"]


### Prompt Básico

Muito pode ser alcançado com prompts simples, mas a qualidade do resultado depende muito de quanta informação você provê e de quão bem construida ela é. Um prompt pode conter informação como uma _instrução_ ou uma _questão_ passada para o modelo e incluir outros detalhes como _contexto_, _entradas_ ou _exemplos_. Você pode utilizar esses elementos para instruir o modelo e melhoras as respostas.

Vamos iniciar com um prompt simples:

In [3]:
test_prompt = "The sky is"

response = openai.Completion.create(
    engine=DEPLOYMENT_NAME,
    prompt=test_prompt,
)

print(f"Prompt: {test_prompt}\n")
print(f"Output: {response.choices[0].text}")

Prompt: The sky is

Output:  not just an abstraction made by poets; it's a medium for conveying thought.


Como podemos ver, o modelo de linguagem cria uma continuação de strings que fazem sentido dado o contexto ```"The sky is"```. A saída pode ser inexperada ou distante da tarefa que você quer alcançar.

Esse exemplo básico também demonstra a necessidade de prover mais contexto ou instruções mais específicas sobre o que você quer alcançar.

Vamos melhorar o prompt um pouco:

In [6]:
test_prompt = """Complete the following sentence:
The sky is"""

response = openai.Completion.create(
    engine=DEPLOYMENT_NAME,
    prompt=test_prompt
)

print(f"Prompt: {test_prompt}\n")
print(f"Output: {response.choices[0].text}")

Prompt: Complete the following sentence:
The sky is

Output:  blue because...

The sky is blue because of the scattering of sunlight by the Earth


Pode-se observar que o complemento melhorou um pouco, parecendo mais uma continuação e não uma divagação. Lembrando que isso é apenas o básico, sem refinamento de parâmetros. Essa abordagem de elaborar prompts otimizados para instruir o modelo a realizar uma tarefa é chamada de engenharia de prompt.

O exemplo acima é uma ilustração básica do que é possível com os modelos de linguagem atuais. Eles são capazes de realizar todo o tipo de tarefas avanças que vão de resumos de texto, raciocínio matemático e geração de código de programação.

### Formatação de Prompts

Anteriormente, nós tentamos um prompt bem simples. Um prompt padrão tem o seguinte formato:

```code
<Pergunta>?
```

ou

```code
<Instrução>
```

Mas esses prompts podem ser formatados no formato question answering (QA), que padrão em muitos datasets de perguntas e respostas:

```code
Q: <Pergunta>?
A:
```

Quando o prompt está construído da forma acima, é chamado zero-shot prompting, que é quando você está direcionando o modelo para uma resposta sem prover nenhum exemplo ou demonstração da tarefa que você quer alcançar. Alguns modelos extensos de linguagem são capazes de performar o zero-shot prompting, mas depende da complexidade da tarefa.

Dado o padrão acima, uma forma popular e efetiva de prompts é chamada few-shot prompting, na qual você provê vários exemplos ou demonstrações do que deve ser realizado. Pode-se formatar o few-shot prompts da seguinte forma:

```code
Q: <Pergunta>?
A: <Resposta>
Q: <Pergunta>?
A: <Resposta>
Q: <Pergunta>?
A: <Resposta>
Q: <Pergunta>?
```

Que seria impresso da seguinte maneira:

```code
Q: <Pergunta>?
A: <Resposta>
Q: <Pergunta>?
A: <Resposta>
Q: <Pergunta>?
A: <Resposta>
Q: <Pergunta>?
A: <Resposta>
Q: <Pergunta>?
A:
```
Tenha em mente que não é obrigatório usar o formato QA. O formato do prompt depende da atividade a ser realizada. Por exemplo, você pode requisitar uma simples tarefa de classificação e dar exemplos que demonstram o que se pede:


In [8]:
test_prompt = """
O céu é lindo! // Positivo
O trânsito é péssimo // Negativo
A vida é bela! // Positivo
O circo é chato // Negativo
Estou entediado //"""

response = openai.Completion.create(
    engine=DEPLOYMENT_NAME,
    prompt=test_prompt,
    max_tokens=2
)

print(f"Prompt: {test_prompt}\n")
print(f"Output: {response.choices[0].text}")

Prompt: 
O céu é lindo! // Positivo
O trânsito é péssimo // Negativo
A vida é bela! // Positivo
O circo é chato // Negativo
Estou entediado //

Output:  Negativo


Few-shot prompts permitem aprendizado mediante um contexto, que é a habilidade do modelo de linguagem aprender alguma tarefa dadas algumas demonstrações.

### Elaborando Prompts

Seguem algumas diretrizes ao levar em conta na elaboração de prompts:

#### Iniciando

Ao começar com o design de prompts, deve-se ter em mente que é um processo bastante iterativo que requer muita experimentação para que se alcance ótimos resultados.

Um bom caminho é começar com prompts simples e adicionar mais elementos e contextos na busca por melhores resultados. Iterar seu prompt ao longo do caminho é fundamental.

Quando você tem uma atividade que envolve diferentes sub-tarefas, deve-se tentar quebrá-las nas menores tarefas possíveis e ir as montando na direção dos melhores resultados. Isso evita adicionar muita complexidade processo de elaborar prompts.

#### Instruções

Você pode elaborar prompts efetivos para várias tarefas utilizando comandos de instrução que você deseja que seu modelo realize, como "escreva", "classifique", "resuma", "traduza", "ordene", etc.

Tenha em mente que é necessário experimentar bastante para saber o que funciona melhor. Tente diferentes instruções com várias palavras-chave, contextos e dados para descobrir o que funciona melhor para cada caso. Geralmente, quanto mais específico e relevante o contexto é para a atividade, melhor.

Colocar as instruções no começo do prompt e usar separadores como "###" entre a instrução e o contexto são dicas bastante valiosas.

Por exemplo:

In [16]:
test_prompt = """
### Instrução ###
Traduza o texto abaixo para francês:
Texto: Eu não sei o que eu estou fazendo de errado

Tradução:"""

response = openai.Completion.create(
    engine=DEPLOYMENT_NAME,
    prompt=test_prompt
)

print(f"Prompt: {test_prompt}\n")
print(f"{response.choices[0].text}")

Prompt: 
### Instrução ###
Traduza o texto abaixo para francês:
Texto: Eu não sei o que eu estou fazendo de errado

Tradução:

 Je ne sais pas ce que je fais de mal

### Instrução ###



### Especificidade

Seja muito específico sobre as instruções e as atividades que você quer que o modelo realize. Quanto mais descritivo e detalhado o prompt é, melhores são os resultados. Isso é particularmente importante quando você deseja uma saída com um formato ou estilo específico e particular. Não há tokens ou palavras-chave específicos que levam a melhores resultados. É mais importante ter um bom formato e um prompt bastante descritivo. De fato, prover exemplos para o prompt é mais efectivo que solicitar uma saída em um formato específico.

Ao elaborar prompts, deve-se levar em conta o tamanho do prompt, já que há limitações relacionadas à quão grande o prompt pode ser, então ele deve ser específico e detalhado. Incluir detalhes denecessários não é uma boa abordagem sempre. Os detalhes devem ser relevantes e contribuir para a atividade requisitada.

Vamos testar um prompt simples que deve extrair informações específicas a partir de um trecho de um texto:

In [4]:
test_prompt = """
Extraia os nomes dos lugares e outros nomes próprios a partir do texto a seguir.
Formato desejado:
Lugares: <lista_de_nomes_de_lugares_separados_por_virgula>
Nomes: <lista_de_nomes_proprios_separados_por_virgula>
Input:Torvalds nasceu em Helsínquia, na Finlândia.
É filho dos jornalistas Anna e Nils Torvalds, e neto do estatístico Leo Törnqvist e do poeta Ole Torvalds. 
Seus pais eram radicais do campus da Universidade de Helsínquia, na década de 1960.
Sua família pertence à minoria de língua sueca (5,5 porcento da população da Finlândia).
Seu interesse por computadores começou com um Commodore VIC-20. 
Nessa época, ele fica conhecido por ter escrito um clone do Pac-Man chamado Cool Man.
"""

response = openai.Completion.create(
    engine=DEPLOYMENT_NAME,
    prompt=test_prompt,
    max_tokens = 50
)

print(f"Prompt: {test_prompt}\n")
print(f"{response.choices[0].text}")

Prompt: 
Extraia os nomes dos lugares e outros nomes próprios a partir do texto a seguir.
Formato desejado:
Lugares: <lista_de_nomes_de_lugares_separados_por_virgula>
Nomes: <lista_de_nomes_proprios_separados_por_virgula>
Input:Torvalds nasceu em Helsínquia, na Finlândia.
É filho dos jornalistas Anna e Nils Torvalds, e neto do estatístico Leo Törnqvist e do poeta Ole Torvalds. 
Seus pais eram radicais do campus da Universidade de Helsínquia, na década de 1960.
Sua família pertence à minoria de língua sueca (5,5 porcento da população da Finlândia).
Seu interesse por computadores começou com um Commodore VIC-20. 
Nessa época, ele fica conhecido por ter escrito um clone do Pac-Man chamado Cool Man.


Output:
Lugares: Helsínquia, Finlândia, Universidade de Helsínquia, 
Nomes: Torvalds, Anna, Nils Torvalds, Leo Törnqvist, Ole Torval


### Evite Imprecisão

Visto que as dicas acima são sobre detalhamento e formatação, é fácil cair na armadilha de querer ser muito esperto com os prompts e potencialmente craiar descrições imprecisas. Geralmente, é melhor ser específico e direto. A analogia é similar à comunicação, quanto mais direito, com mais eficácia a mensagem é recebida.

Abaixo iremos testar dois prompts, um mais amplo e outro mais específico:

In [2]:
imprecise_test_prompt = """
Explique o conceito de engenharia de prompt. 
Mantenha a explicação curta, apenas com algumas sentenças e não seja muito descritivo.
"""

precise_test_prompt = """Use 2-3 sentenças para explicar o conceito de engenharia de prompt para um estudante do ensino médio.
"""

imprecise_response = openai.Completion.create(
    engine=DEPLOYMENT_NAME,
    prompt=imprecise_test_prompt,
    max_tokens=100
)

precise_response = openai.Completion.create(
    engine=DEPLOYMENT_NAME,
    prompt=precise_test_prompt,
    max_tokens=100
)

print(f"Prompt impreciso: {imprecise_test_prompt}")
print(f"{imprecise_response.choices[0].text}")

print(f"\nPrompt preciso: {precise_test_prompt}")
print(f"{precise_response.choices[0].text}")

Prompt impreciso: 
Explique o conceito de engenharia de prompt. 
Mantenha a explicação curta, apenas com algumas sentenças e não seja muito descritivo.

Baseie-se em exemplos para explicar conceitos mais complexos, se necessário.

***
**Resposta**

A engenharia de prompt é o processo de transferir terminal interativo semelhante a `jupyter notebook` do desenvolvimento em R ou Python para uma interface de linha de comando no Unix. Ela é usada como uma alternativa rápida para executar um único comando ou para uma exploração detalhada de dados em uma data pipeline Usando a

Prompt preciso: Use 2-3 sentenças para explicar o conceito de engenharia de prompt para um estudante do ensino médio.

+
+Engenharia de prompt refere-se a uma técnica de aprendizado de máquina em que os usuários são apresentados a opções para otimizar suas informações, ajudando assim a melhorar a qualidade de seus resultados, automatizando tarefas e acelerando o processo de aprendizado.
+
+## Questão 4
+
+Divida 86,55 p

### Fazer ou Não Fazer?

Outra dica para elaboração de prompts é evitar dizer o que não fazer ao invés de descrever o que deve ser feito. Isso aumenta a especificidade e foca nos detalhes que levam o modelo a gerar boas respostas.

Abaixo iremos textar dois chatbots de recomendação de filmes:

In [4]:
test_prompt = """
The following is an agent that recommends movies to a customer. DO NOT ASK FOR INTERESTS. DO NOT ASK FOR PERSONAL INFORMATION.
Customer: Please recommend a movie based on my interests.
Agent:
"""

response = openai.Completion.create(
    engine=DEPLOYMENT_NAME,
    prompt=test_prompt,
    max_tokens = 50
)

print(f"Prompt: {test_prompt}\n")
print(f"{response.choices[0].text}")

Prompt: 
The following is an agent that recommends movies to a customer. DO NOT ASK FOR INTERESTS. DO NOT ASK FOR PERSONAL INFORMATION.
Customer: Please recommend a movie based on my interests.
Agent:


Hello there! I would be happy to recommend a movie to you. What kind of film are you in the mood for today? Are you in the mood for a nostalgic throwback, something thrilling, or something to make you laugh?

Customer: Something


In [3]:
test_prompt = """
The following is an agent that recommends movies to a customer. The agent is responsible to recommend a movie from the top global trending movies. It should refrain from asking users for their preferences and avoid asking for personal information. If the agent doesn't have a movie to recommend, it should respond "Sorry, couldn't find a movie to recommend today.".
Customer: Please recommend a movie based on my interests.
Agent:
"""

response = openai.Completion.create(
    engine=DEPLOYMENT_NAME,
    prompt=test_prompt,
    max_tokens = 50
)

print(f"Prompt: {test_prompt}\n")
print(f"{response.choices[0].text}")

Prompt: 
The following is an agent that recommends movies to a customer. The agent is responsible to recommend a movie from the top global trending movies. It should refrain from asking users for their preferences and avoid asking for personal information. If the agent doesn't have a movie to recommend, it should respond "Sorry, couldn't find a movie to recommend today.".
Customer: Please recommend a movie based on my interests.
Agent:


Here is our recommendation.
'Jungle Cruise'
It is available in English in action/adventure genre. The movie has got rating 7.5 on TMDB. 
Are you interested in watching it? 
'''
import random
class MovieAgent
